In [2]:
import pandas as pd

resultsList = pd.read_csv("./data/results-list.csv")

In [3]:
resultsList

,class_weight,criterion,file,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,presort,random_state,results,splitter
0,NaN,entropy,dataset,NaN,NaN,10,1.000000e-07,1,2,0.0,False,NaN,./data/video-0001-classification.csv,best
1,NaN,mse,dataset,NaN,NaN,10,1.000000e-07,1,2,0.0,False,NaN,./data/video-0001-regression.csv,best
2,NaN,entropy,dataset,NaN,NaN,10,1.000000e-07,2,2,0.0,False,NaN,./data/video-0002-classification.csv,best
3,NaN,mse,dataset,NaN,NaN,10,1.000000e-07,2,2,0.0,False,NaN,./data/video-0002-regression.csv,best
4,NaN,entropy,dataset,NaN,NaN,10,1.000000e-07,4,2,0.0,False,NaN,./data/video-0003-classification.csv,best
5,NaN,mse,dataset,NaN,NaN,10,1.000000e-07,4,2,0.0,False,NaN,./data/video-0003-regression.csv,best
6,NaN,entropy,dataset,NaN,NaN,10,1.000000e-07,6,2,0.0,False,NaN,./data/video-0004-classification.csv,best
7,NaN,mse,dataset,NaN,NaN,10,1.000000e-07,6,2,0.0,False,NaN,./data/video-0004-regression.csv,best
8,NaN,entropy,dataset,NaN,NaN,10,1.000000e-07,8,2,0.0,False,NaN,./data/video-0005-classification.csv,best
9,NaN,mse,dataset,NaN,NaN,10,1.000000e-07,8,2,0.0,False,NaN,./data/video-0005-regression.csv,best


In [5]:
serie = []
for i in resultsList['results']:
    serie.append(i.split("-")[2].split(".")[0])

serie = pd.Series(serie)

resultsList['type'] = serie

accuracy = []
mse = []
for i in resultsList['results']:
    dfTemp = pd.read_csv(i)
    dfMean = dfTemp.groupby(['sr']).mean()
    dfMean['accuracy']= (dfMean['TP']+dfMean['TN']) / (dfMean['TP']+dfMean['TN']+dfMean['FN']+dfMean['FP'])
    accuracy.append(dfMean['accuracy'].mean())
    if 'MSE' in dfMean.columns:
        mse.append(dfMean['MSE'].mean())
    else:
        mse.append(None)
resultsList['mse'] = pd.Series(mse)
resultsList['accuracy'] = pd.Series(accuracy)

idx = resultsList.groupby(["file","type"])["accuracy"].transform(max) == resultsList['accuracy']
resultsList[idx].sort_values(["file"])

,class_weight,criterion,file,max_depth,max_features,max_leaf_nodes,min_impurity_decrease,min_samples_leaf,min_samples_split,min_weight_fraction_leaf,presort,random_state,results,splitter,type,mse,accuracy
122,NaN,entropy,dataset,10.0,NaN,20,1.000000e-07,2,2,0.0,False,NaN,./data/video-0062-classification.csv,best,classification,NaN,0.970486
233,NaN,mse,dataset,20.0,NaN,20,1.000000e-07,4,2,0.0,False,NaN,./data/video-0117-regression.csv,best,regression,0.001809,0.970663


In [17]:
configSet = []
resultsList2 = resultsList[idx][(resultsList['type'] == "classification") | (resultsList['type'] == "regression")].where(pd.notnull(resultsList), None)
for f in resultsList2.file.unique():
    for t in resultsList2.type.unique():
        for index, r in resultsList2.loc[(resultsList["file"]==f) & (resultsList2["type"]==t)].sort_values("accuracy", ascending=False)[:10].iterrows():
            configuration = {}
            for i in r.keys():
                if not i in ["results","type","accuracy","file","mse"]:
                    if not (r["type"] == "regression" and i=="class_weight"):
                        configuration[i] = r[i]
            configSet.append({"file":r["file"],"type":r["type"],"configuration":configuration})

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [18]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import tree

In [24]:
dfConfig = pd.DataFrame(configSet)

d = pd.read_csv("./dataset.csv")
#d = d.sort_values(by="perf")

perf="value that led to label"
label="label (0 if usable and 1 if not)"

srm = 100
srM = d.shape[0]
srs = 100
NSUBS = 10

t=0.5
n = d.shape[1] - 2

for sr in [n,2*n,3*n]:
    shuffle_split = StratifiedShuffleSplit(train_size=sr, n_splits=NSUBS)

    classifier = tree.DecisionTreeClassifier(**dfConfig[dfConfig['type'] == "classification"].configuration.values[0])
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    i = 0
    for train, test in shuffle_split.split(d,d[label]):
        #.fit(d.drop([perf,label],axis=1).iloc[train_index], d[perf].iloc[train_index])
        #probas_ = classifier.fit(d.drop([perf,label],axis=1).iloc[train], d[perf].iloc[train]).predict_proba(d.drop([perf,label],axis=1).iloc[test])
        probas_ = classifier.fit(d.drop([perf,label],axis=1).iloc[train], d[label].iloc[train]).predict_proba(d.drop([perf,label],axis=1).iloc[test])
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(d[label].iloc[test], probas_[:, 1])
        #print(thresholds)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        #plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

        i += 1

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='b',
             label=r'Mean ROC Classification (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    plt.rcParams["figure.figsize"] = [15,9]

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    #plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')

    plt.plot(mean_fpr, tprs_lower, '--', color='b',
             label=r'Mean ROC Classification STD',
             lw=1, alpha=.8)

    plt.plot(mean_fpr, tprs_upper, '--', color='b',
             #label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=1, alpha=.8)
    '''
    d = pd.read_csv(datasetPath+filename+".csv")

    d["label"] = 0
    d.loc[d["perf"] > threshold, "label"] = 1

    X = d.drop(["perf","label"],axis=1,errors="ignore")
    y = d["perf"]
    y_split = d["label"]'''

    classifier = tree.DecisionTreeRegressor(**dfConfig[dfConfig['type'] == "regression"].configuration.values[0])
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)

    i = 0
    for train, test in shuffle_split.split(d,d[label]):
        #.fit(d.drop([perf,label],axis=1).iloc[train], d[perf].iloc[train]).predict_proba(d.drop([perf,label],axis=1).iloc[test])
        probas_ = classifier.fit(d.drop([perf,label],axis=1).iloc[train], d[perf].iloc[train]).predict(d.drop([perf,label],axis=1).iloc[test]) > t
        # Compute ROC curve and area the curve
        fpr, tpr, thresholds = roc_curve(d[perf].iloc[test] > t, probas_)
        #print(thresholds)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        #plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

        i += 1

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    plt.plot(mean_fpr, mean_tpr, color='r',
             label=r'Mean ROC Regression (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=2, alpha=.8)

    plt.rcParams["figure.figsize"] = [15,9]

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    #plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')

    plt.plot(mean_fpr, tprs_lower, '--', color='r',
             label=r'Mean ROC Regression STD',
             lw=1, alpha=.8)

    plt.plot(mean_fpr, tprs_upper, '--', color='r',
             #label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
             lw=1, alpha=.8)

    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='#000000',
             label='Luck', alpha=.8)
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Threshold='+str(t)+"/Sample="+str(sr))
    plt.legend(loc="lower right")
    plt.savefig('ROC_video_'+str(t)+'_'+str(sr)+'.png')
    plt.clf()

/usr/lib64/python3.6/site-packages/sklearn/model_selection/_split.py:1639: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


<Figure size 1080x648 with 0 Axes>

In [27]:
d

,id,background_id,vehicle1_id,vehicle1_trajectory,vehicle1_appearance_change,vehicle1_shadowed,vehicle1_dust,vehicle2_id,vehicle2_trajectory,vehicle2_appearance_change,...,V_mean,V_dev,blur_lvl,static_noise_lvl,dynamic_noise_lvl,frame_instability,line_instability,compression_artefact_lvl,value that led to label,label (0 if usable and 1 if not)
0,1,0,5,0,0,0,0,14,2,0,...,140.907,18.01260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
1,2,0,7,6,0,0,0,14,7,0,...,138.761,30.51420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
2,3,0,8,0,0,0,0,1,0,0,...,141.904,19.21110,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
3,4,0,8,6,0,0,0,14,0,0,...,138.511,27.60950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
4,5,0,2,7,0,0,0,5,5,0,...,141.794,8.11782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
5,6,0,9,4,0,0,0,6,7,0,...,129.561,1.29632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
6,8,0,5,5,0,0,0,11,2,0,...,130.162,21.86900,0.000000,0.410661,0.000000,0.000000,0.000000,0.000000,0.0,0
7,10,0,10,1,0,0,0,5,2,0,...,139.275,21.27440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
8,11,0,13,3,0,0,0,10,1,0,...,143.625,23.53270,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0
9,12,0,8,1,0,0,0,12,0,0,...,129.131,24.54890,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0


In [16]:
dfConfig[dfConfig['type'] == "classification"].configuration.values[0]

{'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10.0,
 'max_features': None,
 'max_leaf_nodes': 20,
 'min_impurity_decrease': 1e-07,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'mse': None,
 'presort': False,
 'random_state': None,
 'splitter': 'best'}